In [0]:
# マウント
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/DIC/group_project

/content/drive/My Drive/DIC/group_project


In [0]:
# ファイル解凍
!unzip image3.zip

Archive:  image3.zip
   creating: image3/
   creating: image3/soda_float/
  inflating: image3/soda_float/soda_flaot__19.jpg  
  inflating: image3/soda_float/soda_flaot__47.jpg  
  inflating: image3/soda_float/soda_flaot__7.jpg  
  inflating: image3/soda_float/soda_flaot__52.jpg  
  inflating: image3/soda_float/soda_flaot__71.jpg  
  inflating: image3/soda_float/soda_flaot__49.jpg  
  inflating: image3/soda_float/soda_flaot__89.jpg  
  inflating: image3/soda_float/soda_flaot__72.jpg  
  inflating: image3/soda_float/soda_flaot__41.jpg  
  inflating: image3/soda_float/soda_flaot__70.jpg  
  inflating: image3/soda_float/soda_flaot__63.jpg  
  inflating: image3/soda_float/soda_flaot__93.jpg  
  inflating: image3/soda_float/soda_flaot__57.jpg  
  inflating: image3/soda_float/soda_flaot__53.jpg  
  inflating: image3/soda_float/soda_flaot__42.jpg  
  inflating: image3/soda_float/soda_flaot__54.jpg  
  inflating: image3/soda_float/soda_flaot__98.jpg  
  inflating: image3/soda_float/soda_flaot__

In [0]:
import os,glob,sys
from tensorflow.python.keras.applications.xception import Xception
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import GaussianNoise
from tensorflow.python.keras import backend as tensorflow_backend
from tensorflow.python.keras.layers import BatchNormalization

Using TensorFlow backend.


In [0]:
import os
from os import listdir
from os.path import isfile, join
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [0]:
# ガウシアンノイズを入れる関数
from numpy.random import *
#from keras.preprocessing.image import load_img, img_to_array, array_to_img
import cv2

import datetime


def add_noise(my_path='/content/drive/My Drive/DIC/group_project/image3/train/cclemon/', iter_num=1):
    
    size = 224

    files_name = [f for f in listdir(my_path) if isfile(join(my_path, f))]

    for _ in range(iter_num):
        for file in files_name:
            image = cv2.imread(my_path + file)
            image = image + randint(1,255,(image.shape[0], image.shape[1], 3)) / 5
            #x = x.reshape((1,) + x.shape)
            cv2.imwrite(my_path + file[0:6] + str(datetime.datetime.now().time())[-4:] +'.jpg', image)


In [0]:
add_noise(my_path='/content/drive/My Drive/DIC/group_project/image3/train/cclemon/')
add_noise(my_path='/content/drive/My Drive/DIC/group_project/image3/train/cocacola/')
add_noise(my_path='/content/drive/My Drive/DIC/group_project/image3/train/fanta_litchi/')
add_noise(my_path='/content/drive/My Drive/DIC/group_project/image3/train/namacha/')
add_noise(my_path='/content/drive/My Drive/DIC/group_project/image3/train/soda_float/')

In [0]:
my_path = 'image2/'

In [0]:
#オーグメンテーションする関数
def augmentation(dir_path, augment_num=5):
    
    
    """
    note : 指定ディレクトリ内の,指定頭文字で始まるファイルを指定枚数オーグメントする
    ----------
    dir_path : フォルダパス
    initial_letter : augmentしたいファイル名の頭文字
    aument_num : augmentしたい枚数
    ----------
    """ 
    datagen = ImageDataGenerator(rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest'
    )
    
    
    for i, fold in tqdm(enumerate(listdir(my_path))):
      if fold != 'train' and fold !=  'test':
        for file in listdir(my_path+fold):
          if not os.path.exists(my_path+fold+'/augmentation'):
            os.mkdir(my_path+fold+'/augmentation')
          if file != 'augmentation':
              img = load_img(dir_path + fold + '/'+ file)
              x = img_to_array(img)
              x = x.reshape((1,) + x.shape) 

              i = 0
              for batch in datagen.flow(x, save_to_dir=dir_path+fold+'/augmentation', save_prefix=fold, save_format="jpg"):
                  i += 1
                  if i > augment_num:
                      break

                  else:
                      pass

In [0]:
augmentation(my_path)

# exceptionモデル作成

In [0]:
import numpy as np
import scipy # 画像のリサイズ用

from keras.models import Sequential, Model, load_model, Input
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam, RMSprop
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

# 訓練済みのXceptionモデルとその前処理用関数
from keras.applications.xception import Xception, preprocess_input

In [0]:
#分類するクラスを設定する
#classes = ['coca_img', 'tea_img', 'soda_img', 'fanta_img', 'cclemon_img']
classes = ['soda_float', 'fanta_litchi', 'cclemon', 'namacha','cocacola']
#classes = ['class0', 'class1', 'class2', 'class3']
nb_classes = len(classes)
#pet_dict = {'lemon':0,'cocacola':1,'fanta_litchi':2,'namacha':3,'soda_float':4}

# 学習データと検証データのファイル場所を設定する
train_data_dir = 'image3/train'
validation_data_dir = 'image3/test'

# イメージサイズを設定する
img_width, img_height = 224, 224

In [0]:
#trainとvalidに分けてgeneratorを作成
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1.0 / 255, 
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2, 
                                   shear_range=5,
                                   brightness_range=[0.7, 1.0],
                                   horizontal_flip=True,)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [0]:
train_generator = train_datagen.flow_from_directory(
  train_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=16)
 
validation_generator = validation_datagen.flow_from_directory(
  validation_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=16)

Found 1744 images belonging to 5 classes.
Found 96 images belonging to 5 classes.


#モデル作成
K.clear_session()
# インプットデータ型を設定
input_tensor = Input(shape=(img_width, img_height, 3))
# ResNetを 活用
xception = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3))
top_model = Sequential()
top_model.add(Flatten(input_shape=xception.output_shape[1:]))
top_model.add(Dense(nb_classes, activation='softmax'))
model = Model(input=xception.input, output=top_model(xception.output))

set_trainable = False
for layer in xception.layers:
    if layer.name == 'block14_sepconv1':
        # 一切の学習を行わない場合
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
# シーケンシャルモデルで使用
K.clear_session()
# インプットデータ型を設定
input_tensor = Input(shape=(img_width, img_height, 3))
# ResNetを活用
xception = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3))
model2 = Sequential()
model2.add(xception)
model2.add(Flatten())
model2.add(Dense(nb_classes, activation='softmax'))

set_trainable = False
for layer in xception.layers:
    if layer.name == 'block14_sepconv1':
        # 一切の学習を行わない場合
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
from keras import optimizers
model2.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

In [0]:
# 学習データと検証データの数を設定
nb_train_samples = 100
nb_validation_samples = 50
mc = ModelCheckpoint("xception_model_07.h5", monitor="val_loss", save_best_only=True, verbose=1)
es = EarlyStopping(monitor='val_loss', patience=5)

history = model2.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    nb_epoch=30,
    callbacks=[mc, es],
    validation_data=validation_generator,
    validation_steps  =nb_validation_samples)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=100, callbacks=[<keras.ca..., validation_data=<keras_pre..., validation_steps=50, epochs=30)`
  if sys.path[0] == '':


Epoch 1/30
100/100 [==============================] - 42s 421ms/step - loss: 0.8629 - acc: 0.8875 - val_loss: 1.1731e-06 - val_acc: 1.0000

Epoch 00001: val_loss improved from inf to 0.00000, saving model to xception_model_07.h5
Epoch 2/30
100/100 [==============================] - 41s 406ms/step - loss: 0.0497 - acc: 0.9875 - val_loss: 5.8398e-07 - val_acc: 1.0000

Epoch 00002: val_loss improved from 0.00000 to 0.00000, saving model to xception_model_07.h5
Epoch 3/30
100/100 [==============================] - 40s 402ms/step - loss: 0.0595 - acc: 0.9887 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00003: val_loss improved from 0.00000 to 0.00000, saving model to xception_model_07.h5
Epoch 4/30
100/100 [==============================] - 39s 394ms/step - loss: 0.1182 - acc: 0.9856 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/30
100/100 [==============================] - 38s 384ms/step - loss: 0.0315 - acc: 0.9950 - val_loss: 1.19

KeyboardInterrupt: ignored

In [0]:
model = load_model('xception_model_07.h5')

In [0]:
listdir('image2/test/soda_float/')

['soda_float__20190709194553.jpg', 'soda_float__20190709194545.jpg']

In [0]:
#namachaを予測
from keras.preprocessing import image

image_data = '/content/drive/My Drive/DIC/group_project/image3/test/namacha/namacha__81.jpg'
namacha_img = image.load_img(image_data, target_size=(img_width, img_height))
x = image.img_to_array(namacha_img)
x = np.expand_dims(x, axis=0)
x = x / 255
pred = model.predict(x)[0]

 
# 予測確率を出力
top = 5
top_indices = pred.argsort()[-top:][::-1]
result = [(classes[i], pred[i]) for i in top_indices]
print(result)

[('namacha', 1.0), ('cocacola', 0.0), ('cclemon', 0.0), ('fanta_litchi', 0.0), ('soda_float', 0.0)]


In [0]:
#ladyboyを予測する場合
from keras.preprocessing import image

image_data = '/content/drive/My Drive/DIC/group_project/image2/ladyboy/ladyboy.jpeg'
soda_img = image.load_img(image_data, target_size=(img_width, img_height))
x = image.img_to_array(soda_img)
x = np.expand_dims(x, axis=0)
x = x / 255
pred = model2.predict(x)[0]

 
# 予測確率を出力
top = 5
top_indices = pred.argsort()[-top:][::-1]
result = [(classes[i], pred[i]) for i in top_indices]
print(result)

[('cclemon', 0.89594984), ('namacha', 0.09858679), ('cocacola', 0.005463424), ('fanta_litchi', 2.5556605e-11), ('soda_float', 1.123386e-17)]


In [0]:
from os import listdir
from os.path import isfile, join
import time

from keras.models import Model


def output_logit(layer_name='dense_12', test_image_path='test_imageのコピー/'):
    """
    note : 特定層においてロジットの出力をarrayにて出力する関数
    ----------
    attribute
    layer_name : layer
    test_image_path : test_image(predictする画像)が格納されているフォルダ
    """
    # 画像ファイルの取り出し
    test_images = [f for f in listdir(test_image_path) if isfile(join(test_image_path, f))]
    
    hidden_layer_model = Model(inputs=model2.input, outputs=model2.get_layer('dense_1').output)
    
    for test_image in test_images:
    
        img = cv2.imread(test_image_path + test_image)
        img = cv2.resize(img, (224, 224))
        target = np.reshape(img, (1, img.shape[0], img.shape[1], img.shape[2])).astype('float') / 255.0
        array = hidden_layer_model.predict(target)
        print('file_name :', test_image)
        print('array :', array)
        print('std :', np.std(array), 'var :', np.var(array))
        print('--------------------------------------------------------------------------------------')


In [0]:
output_logit('sequential_7', '/content/drive/My Drive/DIC/group_project/image3/test/cclemon/')

file_name : cclemon__81.jpg
array : [[3.5928857e-01 4.0284514e-15 6.4071143e-01 4.8495674e-10 1.8788657e-10]]
std : 0.26061445 var : 0.06791989
--------------------------------------------------------------------------------------
file_name : cclemon__83.jpg
array : [[5.6964986e-06 3.0863346e-14 9.9999428e-01 6.7266226e-10 7.6170695e-11]]
std : 0.39999717 var : 0.15999773
--------------------------------------------------------------------------------------
file_name : cclemon__85.jpg
array : [[5.1168037e-08 9.2192970e-14 1.0000000e+00 7.0181810e-10 1.3480293e-10]]
std : 0.4 var : 0.16000001
--------------------------------------------------------------------------------------
file_name : cclemon__84.jpg
array : [[3.7267421e-07 6.0435337e-13 9.9999964e-01 5.9819274e-09 1.0776701e-09]]
std : 0.39999986 var : 0.15999988
--------------------------------------------------------------------------------------
file_name : cclemon__82.jpg
array : [[9.9750441e-01 9.3346936e-15 2.4956544e-03 2.6

In [0]:
import cv2
output_logit('sequential_7', '/content/drive/My Drive/DIC/group_project/image2/ladyboy/')

file_name : ladyboy.jpeg
array : [[5.4600400e-01 6.9185182e-09 3.9012133e-07 4.5305023e-01 9.4542489e-04]]
std : 0.24632327 var : 0.06067515
--------------------------------------------------------------------------------------
file_name : ladyboy2.jpeg
array : [[9.9401855e-01 9.9954605e-07 7.7698141e-06 3.0132948e-04 5.6713135e-03]]
std : 0.39701515 var : 0.15762104
--------------------------------------------------------------------------------------


In [0]:
open('xception_model6.json',"w").write(model.to_json())

# 学習済みの重みを保存
model.save_weights('xception_model6_weight.hdf5')

ロジット値のstdなどの変化がladyboyでもあまり変わらない。
訓練が足りないかも

# 2stage目に別の分類モデルを入れる

In [0]:
image_data = '/content/drive/My Drive/DIC/group_project/image3/test/'
listdir(image_data)

['namacha', 'soda_float', 'fanta_litchi', 'cclemon', 'cocacola']

In [0]:
#1stmodelがpredした値と、rgbに関する情報をまとめる

pet_fold = '/content/drive/My Drive/DIC/group_project/image3/test/'
fold_names = listdir(pet_fold)
kakuritu_df_list = pd.DataFrame([])
label_list = []
for fold in fold_names:
  file_names = listdir(pet_fold+fold+'/')
  for file in file_names:
    img = image.load_img(pet_fold+fold+'/'+file, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x_ = np.expand_dims(x, axis=0)
    x_ = x_ / 255
    pred = model2.predict(x_)[0] #predを変数へ
    rgb_mean = np.mean(np.mean(x,0),0) / 255 # 画像からrgbの平均値を抽出
    rgb_rate = rgb_mean / rgb_mean.sum() # rgbそれぞれの割合を算出
    new_feature = np.hstack([pred, rgb_mean,rgb_rate]).reshape(1,-1) #predとrgb平均、rgb割合をまとめてstack
    df_kakuritu = pd.DataFrame(new_feature) #dfへ
    kakuritu_df_list = pd.concat([kakuritu_df_list, df_kakuritu])
    label_list.append(fold) #画像の正解ラベル(フォルダ名)を同様の並びでlabel_listとして格納
label_list = pd.DataFrame(label_list)
kakuritu_df_list.columns = 'namacha','soda_float','fanta_litchi','cclemon','cocacola','r','g','b','r_rate','g_rate','b_rate'

In [0]:
kakuritu_df_list

,namacha,soda_float,fanta_litchi,cclemon,cocacola,r,g,b,r_rate,g_rate,b_rate
0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.120041,0.123769,0.112457,0.336940,0.347405,0.315655
0,0.000000e+00,2.998585e-38,1.149757e-35,1.000000e+00,0.000000e+00,0.116259,0.119511,0.107961,0.338227,0.347689,0.314084
0,5.024981e-38,2.822518e-35,2.126980e-31,1.000000e+00,1.112828e-35,0.115610,0.119419,0.107919,0.337108,0.348213,0.314679
0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.120303,0.123930,0.112679,0.337067,0.347229,0.315705
0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.121022,0.123470,0.111950,0.339528,0.346396,0.314076
0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.120585,0.123687,0.112238,0.338236,0.346940,0.314824
0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.121211,0.123654,0.111629,0.340008,0.346862,0.313130
0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.121656,0.124048,0.112127,0.339982,0.346667,0.313351
0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.118779,0.120550,0.109124,0.340874,0.345958,0.313168
0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.121176,0.123560,0.112017,0.339664,0.346346,0.313991


In [0]:
label_list

,0
0,namacha
1,namacha
2,namacha
3,namacha
4,namacha
5,namacha
6,namacha
7,namacha
8,namacha
9,namacha
